# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### --PENDING-- US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats --PENDING--
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats --PENDING--

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'07-10-2020'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'07-10-2020'

In [4]:
# Check out the first 5 rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
0,45001.0,Abbeville,South Carolina,US,2020-07-11 04:34:26,34.223334,-82.461707,141,1,0,140.0,"Abbeville, South Carolina, US",574.876667,0.709220
1,22001.0,Acadia,Louisiana,US,2020-07-11 04:34:26,30.295065,-92.414197,1232,44,0,1188.0,"Acadia, Louisiana, US",1985.655573,3.571429
2,51001.0,Accomack,Virginia,US,2020-07-11 04:34:26,37.767072,-75.632346,1042,14,0,1028.0,"Accomack, Virginia, US",3224.408962,1.343570
3,16001.0,Ada,Idaho,US,2020-07-11 04:34:26,43.452658,-116.241552,3790,25,0,3765.0,"Ada, Idaho, US",786.981376,0.659631
4,19001.0,Adair,Iowa,US,2020-07-11 04:34:26,41.330756,-94.471059,17,0,0,17.0,"Adair, Iowa, US",237.695749,0.000000


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,7/1/20,7/2/20,7/3/20,7/4/20,7/5/20,7/6/20,7/7/20,7/8/20,7/9/20,7/10/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,31836,32022,32324,32672,32951,33190,33384,33594,33908,34194
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,2580,2662,2752,2819,2893,2964,3038,3106,3188,3278
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,14272,14657,15070,15500,15941,16404,16879,17348,17808,18242


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,7/1/20,7/2/20,7/3/20,7/4/20,7/5/20,7/6/20,7/7/20,7/8/20,7/9/20,7/10/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,774,807,819,826,864,898,920,936,957,971
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,65,69,72,74,76,79,81,83,83,85
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,920,928,937,946,952,959,968,978,988,996


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,7/1/20,7/2/20,7/3/20,7/4/20,7/5/20,7/6/20,7/7/20,7/8/20,7/9/20,7/10/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,15651,16041,17331,19164,19366,20103,20179,20700,20847,20882
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,1516,1559,1592,1637,1657,1702,1744,1791,1832,1875
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,10040,10342,10832,11181,11492,11884,12094,12329,12637,13124


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,7/1/20,7/2/20,7/3/20,7/4/20,7/5/20,7/6/20,7/7/20,7/8/20,7/9/20,7/10/20
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,...,0,0,0,0,0,0,0,0,0,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,...,267,280,280,280,280,301,303,307,309,310
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,...,30,31,31,31,31,31,31,31,31,31


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,7/1/20,7/2/20,7/3/20,7/4/20,7/5/20,7/6/20,7/7/20,7/8/20,7/9/20,7/10/20
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,...,0,0,0,0,0,0,0,0,0,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,...,5,5,5,5,5,5,5,5,5,5
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,...,2,2,2,2,2,2,2,2,2,2


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
1683,1085.0,Lowndes,Alabama,US,2020-07-11 04:34:26,32.159728,-86.651584,480,22,0,458.0,"Lowndes, Alabama, US",4935.225170,4.583333
108,1001.0,Autauga,Alabama,US,2020-07-11 04:34:26,32.539527,-86.644082,662,14,0,648.0,"Autauga, Alabama, US",1184.914711,2.114804
1959,1103.0,Morgan,Alabama,US,2020-07-11 04:34:26,34.455006,-86.854759,1241,5,0,1236.0,"Morgan, Alabama, US",1036.940482,0.402901


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,3184573,134092,983185
Brazil,1800827,70398,1217361
India,820916,22123,515386


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,3184573,134092,983185,2067296,2020-07-11 04:34:26,38.322678,-91.732682
Brazil,1800827,70398,1217361,513068,2020-07-11 04:34:26,-12.669522,-48.480493
India,820916,22123,515386,283407,2020-07-11 04:34:26,23.356511,82.081064
Russia,712863,11000,488234,213629,2020-07-11 04:34:26,54.546312,62.120860
Peru,319646,11500,210638,97508,2020-07-11 04:34:26,-10.637324,-75.277744


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
New York,400299,32331,0
California,310885,6955,0
Texas,244756,3062,0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
New York,400299,32331,0,367968,2020-07-11 04:34:26,42.671593,-75.579694
California,310885,6955,0,303930,2020-07-11 04:34:26,37.778248,-120.728666
Texas,244756,3062,0,241694,2020-07-11 04:34:26,31.648720,-98.600835
Florida,244151,4102,0,240049,2020-07-11 04:34:26,28.940755,-82.700744
New Jersey,174628,15479,0,159149,2020-07-11 04:34:26,40.286441,-74.622450


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
New York,New York City,218403,23267,0
California,Los Angeles,127439,3744,0
Illinois,Cook,94605,4693,0
Arizona,Maricopa,76328,1049,0
Florida,Miami-Dade,58341,1118,0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
New York,New York City,218403,23267,0,195136,2020-07-11 04:34:26,40.767273,-73.971526,36061.0
California,Los Angeles,127439,3744,0,123695,2020-07-11 04:34:26,34.308284,-118.228241,6037.0
Illinois,Cook,94605,4693,0,89912,2020-07-11 04:34:26,41.841448,-87.816588,17031.0
Arizona,Maricopa,76328,1049,0,75279,2020-07-11 04:34:26,33.348359,-112.491815,4013.0
Florida,Miami-Dade,58341,1118,0,57223,2020-07-11 04:34:26,25.611236,-80.551706,12086.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-07-08,33594,3106,17348,855,386,70,87030,29820,9056,18513,...,288511,974,11092,8008,369,5029,10,1318,1895,885
2020-07-09,33908,3188,17808,855,396,73,90693,30346,9374,18615,...,289154,977,11564,8372,369,5220,10,1356,1895,885
2020-07-10,34194,3278,18242,855,458,74,94060,30903,9553,18709,...,289678,985,12027,8803,370,5551,10,1380,1895,942


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-07-08,936,83,978,52,21,3,1694,521,106,706,...,44602,29,45,75,0,20,1,351,42,9
2020-07-09,957,83,988,52,22,3,1720,535,106,706,...,44687,29,51,80,0,24,1,361,42,9
2020-07-10,971,85,996,52,23,3,1774,546,107,706,...,44735,29,54,83,0,27,1,364,42,13


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-07-08,20700,1791,12329,802,117,23,36502,17427,7573,16721,...,1378,871,7060,2100,347,494,8,595,1348,206
2020-07-09,20847,1832,12637,802,117,57,38313,18000,7576,16758,...,1378,878,7287,2544,347,525,8,619,1348,206
2020-07-10,20882,1875,13124,803,117,57,38984,18709,7733,16808,...,1378,886,7530,2671,350,536,8,630,1348,320


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,7/1/20,7/2/20,7/3/20,7/4/20,7/5/20,7/6/20,7/7/20,7/8/20,7/9/20,7/10/20
3147,84080001,US,USA,840,80001.0,Out of AL,Alabama,US,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
52,84001095,US,USA,840,1095.0,Marshall,Alabama,US,34.369760,-86.304867,...,1537,1622,1680,1702,1711,1724,1770,1815,1866,1904
51,84001093,US,USA,840,1093.0,Marion,Alabama,US,34.136974,-87.887042,...,203,206,214,219,224,233,236,241,245,267
50,84001091,US,USA,840,1091.0,Marengo,Alabama,US,32.247676,-87.787962,...,286,299,300,302,302,312,317,320,330,342
48,84001087,US,USA,840,1087.0,Macon,Alabama,US,32.387599,-85.692677,...,170,177,181,185,194,199,199,202,206,212


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-07-08,46962,1222,0,108614,25246,292560,35099,47108,12462,49,...,55986,224929,26755,1256,122,67375,37941,3707,33154,1740
2020-07-09,49174,1268,0,112671,26052,302484,35507,47209,12531,49,...,57591,236541,27356,1272,144,67988,38581,3826,33908,1769
2020-07-10,50508,1323,0,116892,26803,310885,36173,47287,12652,49,...,59546,244756,28223,1277,153,68931,39218,3983,34753,1790


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-07-08         643    1039     362  198    258     369    637     403   
2020-07-09         653    1114     367  208    273     370    643     436   
2020-07-10         662    1170     377  215    281     376    645     456   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-07-08          647      111  ...      76      4       27        6   
2020-07-09          657      112  ...      80      4       27        6   
2020-07-10          664      115  ...      81      4       29        6   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-07-08            124   149   192          0       43      1  
2020-07-09            126   149   198          0       43      1  
2020-07-10            128   150   200          0       42      1  

[3 rows x 3254 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-07-08,1058,17,0,1963,305,6718,1704,4343,515,0,...,685,2875,201,56,6,1905,1394,95,807,21
2020-07-09,1068,17,0,2038,309,6859,1706,4348,517,0,...,710,3006,205,56,6,1937,1409,95,809,21
2020-07-10,1104,17,0,2082,313,6955,1724,4348,517,0,...,723,3062,207,56,6,1958,1424,95,814,21


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-07-08          12       9       2    1      1      10     28       5   
2020-07-09          13      10       2    1      1      10     28       5   
2020-07-10          14      11       2    1      1      10     29       5   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-07-08           27        7  ...       0      0        0        0   
2020-07-09           27        7  ...       0      0        0        0   
2020-07-10           27        7  ...       0      0        0        0   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-07-08              0     0     0         20        0      0  
2020-07-09              0     0     0         20        0      0  
2020-07-10              0     0     0         20        0      0  

[3 rows x 3254 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-07-08,0.006290,0.022383,0.027786,0.0,0.000000,0.000000,0.043200,0.018269,0.019131,0.004994,...,0.002213,0.009326,0.023436,0.040946,0.00000,0.082204,0.0,0.016191,0.0,0.124524
2020-07-09,0.009347,0.026401,0.026516,0.0,0.025907,0.042857,0.042089,0.017639,0.035115,0.005510,...,0.002229,0.003080,0.042553,0.045455,0.00000,0.037980,0.0,0.028832,0.0,0.000000
2020-07-10,0.008435,0.028231,0.024371,0.0,0.156566,0.013699,0.037125,0.018355,0.019095,0.005050,...,0.001812,0.008188,0.040038,0.051481,0.00271,0.063410,0.0,0.017699,0.0,0.064407


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-07-08,0.017391,0.024691,0.010331,0.0,0.000000,0.0,0.030414,0.035785,0.000000,0.0,...,0.002833,0.0,0.097561,0.056338,NaN,0.111111,0.0,0.008621,0.0,0.000000
2020-07-09,0.022436,0.000000,0.010225,0.0,0.047619,0.0,0.015348,0.026871,0.000000,0.0,...,0.001906,0.0,0.133333,0.066667,NaN,0.200000,0.0,0.028490,0.0,0.000000
2020-07-10,0.014629,0.024096,0.008097,0.0,0.045455,0.0,0.031395,0.020561,0.009434,0.0,...,0.001074,0.0,0.058824,0.037500,NaN,0.125000,0.0,0.008310,0.0,0.444444


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-07-08,0.025819,0.026950,0.019431,0.002500,0.0,0.000000,0.212893,0.030756,0.011487,0.002098,...,0.002182,0.006936,0.036559,0.000000,0.014620,0.000000,0.0,0.006768,0.0,0.024876
2020-07-09,0.007101,0.022892,0.024982,0.000000,0.0,1.478261,0.049614,0.032880,0.000396,0.002213,...,0.000000,0.008037,0.032153,0.211429,0.000000,0.062753,0.0,0.040336,0.0,0.000000
2020-07-10,0.001679,0.023472,0.038538,0.001247,0.0,0.000000,0.017514,0.039389,0.020723,0.002984,...,0.000000,0.009112,0.033347,0.049921,0.008646,0.020952,0.0,0.017771,0.0,0.553398


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-07-08,0.025707,0.035593,NaN,0.033494,0.029945,0.030097,0.013046,0.001595,0.003867,0.0,...,0.046194,0.041213,0.027734,0.001595,0.051724,0.009515,0.013923,0.057632,0.018368,0.018139
2020-07-09,0.047102,0.037643,NaN,0.037352,0.031926,0.033921,0.011624,0.002144,0.005537,0.0,...,0.028668,0.051625,0.022463,0.012739,0.180328,0.009098,0.016868,0.032101,0.022742,0.016667
2020-07-10,0.027128,0.043375,NaN,0.037463,0.028827,0.027773,0.018757,0.001652,0.009656,0.0,...,0.033946,0.034730,0.031693,0.003931,0.062500,0.013870,0.016511,0.041035,0.024920,0.011871


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-07-08      0.011006  0.060204  0.014006  0.010204  0.027888  0.002717   
2020-07-09      0.015552  0.072185  0.013812  0.050505  0.058140  0.002710   
2020-07-10      0.013783  0.050269  0.027248  0.033654  0.029304  0.016216   

Province_State                                          ...   Wyoming         \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park Platte   
2020-07-08      0.007911  0.041344  0.006221  0.077670  ...  0.085714    0.0   
2020-07-09      0.009419  0.081886  0.015456  0.009009  ...  0.052632    0.0   
2020-07-10      0.003110  0.045872  0.010654  0.026786  ...  0.012500    0.0   

Province_State                                                               \
Admin2          Sheridan Sublette Sweetwater     Teton     Uinta Unassigned   
2020-07-08      0.000000      0.0   0.016393  0.020548  0.010526        NaN   
2020-07-09      0.000000      0.0   0.016129  0.000000  0.031250        NaN   
2020-07-10      0.074074      0.0   0.015873  0.006711  0.010101        NaN   

Province_State                   
Admin2          Washakie Weston  
2020-07-08      0.075000    0.0  
2020-07-09      0.000000    0.0  
2020-07-10     -0.023256    0.0  

[3 rows x 3254 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-07-08,0.024201,0.0,NaN,0.018682,0.013289,0.022060,0.004717,0.001153,0.001946,NaN,...,0.030075,0.034917,0.036082,0.0,0.0,0.012759,0.007225,0.0,0.002484,0.0
2020-07-09,0.009452,0.0,NaN,0.038207,0.013115,0.020988,0.001174,0.001151,0.003883,NaN,...,0.036496,0.045565,0.019900,0.0,0.0,0.016798,0.010760,0.0,0.002478,0.0
2020-07-10,0.033708,0.0,NaN,0.021590,0.012945,0.013996,0.010551,0.000000,0.000000,NaN,...,0.018310,0.018629,0.009756,0.0,0.0,0.010842,0.010646,0.0,0.006180,0.0


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State   Alabama                                                  \
Admin2           Autauga   Baldwin Barbour Bibb Blount Bullock    Butler   
2020-07-08      0.000000  0.000000     0.0  0.0    0.0     0.0  0.000000   
2020-07-09      0.083333  0.111111     0.0  0.0    0.0     0.0  0.000000   
2020-07-10      0.076923  0.100000     0.0  0.0    0.0     0.0  0.035714   

Province_State                            ... Wyoming                  \
Admin2         Calhoun Chambers Cherokee  ...    Park Platte Sheridan   
2020-07-08         0.0      0.0      0.0  ...     NaN    NaN      NaN   
2020-07-09         0.0      0.0      0.0  ...     NaN    NaN      NaN   
2020-07-10         0.0      0.0      0.0  ...     NaN    NaN      NaN   

Province_State                                                             
Admin2         Sublette Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-07-08          NaN        NaN   NaN   NaN        0.0      NaN    NaN  
2020-07-09          NaN        NaN   NaN   NaN        0.0      NaN    NaN  
2020-07-10          NaN        NaN   NaN   NaN        0.0      NaN    NaN  

[3 rows x 3254 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-07-08,0.006652,0.023859,0.028277,7.077668e-10,0.032167,0.003572,0.039487,0.016564,0.018058,0.004596,...,0.001988,0.006815,0.031588,0.040381,0.004932,0.075653,8.477105e-07,0.014189,0.040433,0.089246
2020-07-09,0.007999,0.025130,0.027397,3.538834e-10,0.029037,0.023214,0.040788,0.017101,0.026586,0.005053,...,0.002108,0.004948,0.037071,0.042918,0.002466,0.056816,4.238553e-07,0.021510,0.020216,0.044623
2020-07-10,0.008217,0.026680,0.025884,1.769417e-10,0.092801,0.018457,0.038957,0.017728,0.022841,0.005051,...,0.001960,0.006568,0.038554,0.047200,0.002588,0.060113,2.119276e-07,0.019605,0.010108,0.054515


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-07-08,0.023671,0.027741,0.009412,2.337437e-09,0.031019,2.605134e-26,0.034296,0.027874,0.001202,0.000098,...,0.002504,0.004629,0.096850,0.051044,NaN,0.104125,0.0,0.01025,0.101207,0.020254
2020-07-09,0.023054,0.013870,0.009818,1.168719e-09,0.039319,1.302567e-26,0.024822,0.027373,0.000601,0.000049,...,0.002205,0.002315,0.115091,0.058855,NaN,0.152063,0.0,0.01937,0.050604,0.010127
2020-07-10,0.018841,0.018983,0.008958,5.843593e-10,0.042387,6.512836e-27,0.028109,0.023967,0.005018,0.000025,...,0.001639,0.001157,0.086957,0.048178,NaN,0.138531,0.0,0.01384,0.025302,0.227286


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-07-08,0.024681,0.025488,0.022267,0.001260,0.023053,0.000355,0.130714,0.028650,0.008418,0.002063,...,0.001132,0.007827,0.029482,0.002260,0.008531,0.005506,4.967054e-09,0.018613,0.000115,0.030396
2020-07-09,0.015891,0.024190,0.023624,0.000630,0.011526,0.739308,0.090164,0.030765,0.004407,0.002138,...,0.000566,0.007932,0.030817,0.106844,0.004265,0.034130,2.483527e-09,0.029474,0.000058,0.015198
2020-07-10,0.008785,0.023831,0.031081,0.000938,0.005763,0.369654,0.053839,0.035077,0.012565,0.002561,...,0.000283,0.008522,0.032082,0.078383,0.006455,0.027541,1.241763e-09,0.023623,0.000029,0.284298


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-07-08,0.023992,0.028975,NaN,0.034763,0.023278,0.033988,0.011066,0.001845,0.007430,1.639089e-35,...,0.035486,0.042104,0.024703,0.002442,0.041694,0.009092,0.015853,0.042912,0.017364,0.019517
2020-07-09,0.035547,0.033309,NaN,0.036058,0.027602,0.033954,0.011345,0.001994,0.006484,8.195447e-36,...,0.032077,0.046864,0.023583,0.007590,0.111011,0.009095,0.016361,0.037507,0.020053,0.018092
2020-07-10,0.031338,0.038342,NaN,0.036760,0.028214,0.030864,0.015051,0.001823,0.008070,4.097723e-36,...,0.033012,0.040797,0.027638,0.005761,0.086755,0.011483,0.016436,0.039271,0.022487,0.014981


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-07-08      0.021152  0.063454  0.012672  0.012795  0.029404  0.001156   
2020-07-09      0.018352  0.067820  0.013242  0.031650  0.043772  0.001933   
2020-07-10      0.016067  0.059044  0.020245  0.032652  0.036538  0.009075   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-07-08      0.004784  0.043695  0.007863  0.058528  ...  0.069566   
2020-07-09      0.007102  0.062791  0.011660  0.033768  ...  0.061099   
2020-07-10      0.005106  0.054331  0.011157  0.030277  ...  0.036799   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-07-08      0.002286  0.000137  0.000051   0.041189  0.018613  0.010250   
2020-07-09      0.001143  0.000068  0.000025   0.028659  0.009307  0.020750   
2020-07-10      0.000572  0.037071  0.000013   0.022266  0.008009  0.015425   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2020-07-08            NaN  0.043910 -0.061523  
2020-07-09            NaN  0.021955 -0.030762  
2020-07-10            NaN -0.000650 -0.015381  

[3 rows x 3254 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-07-08,0.019176,0.018845,NaN,0.024361,0.017638,0.018316,0.002412,0.000704,0.002036,NaN,...,0.022923,0.029938,0.029075,8.669771e-09,1.127588e-18,0.010831,0.007731,0.001669,0.004230,0.012500
2020-07-09,0.014314,0.009422,NaN,0.031284,0.015376,0.019652,0.001793,0.000928,0.002960,NaN,...,0.029710,0.037752,0.024488,4.334885e-09,5.637941e-19,0.013814,0.009246,0.000835,0.003354,0.006250
2020-07-10,0.024011,0.004711,NaN,0.026437,0.014161,0.016824,0.006172,0.000464,0.001480,NaN,...,0.024010,0.028191,0.017122,2.167443e-09,2.818970e-19,0.012328,0.009946,0.000417,0.004767,0.003125


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                                \
Admin2           Autauga       Baldwin   Barbour Bibb Blount   Bullock   
2020-07-08      0.000743  4.470043e-16  0.015625  0.0    0.0  0.003378   
2020-07-09      0.042038  5.555556e-02  0.007812  0.0    0.0  0.001689   
2020-07-10      0.059481  7.777778e-02  0.003906  0.0    0.0  0.000845   

Province_State                                                      ...  \
Admin2            Butler       Calhoun      Chambers      Cherokee  ...   
2020-07-08      0.000579  3.854434e-06  5.240299e-09  2.735768e-06  ...   
2020-07-09      0.000290  1.927217e-06  2.620149e-09  1.367884e-06  ...   
2020-07-10      0.018002  9.636084e-07  1.310075e-09  6.839420e-07  ...   

Province_State Wyoming                                                  \
Admin2            Park Platte Sheridan Sublette Sweetwater Teton Uinta   
2020-07-08         NaN    NaN      NaN      NaN        NaN   NaN   NaN   
2020-07-09         NaN    NaN      NaN      NaN        NaN   NaN   NaN   
2020-07-10         NaN    NaN      NaN      NaN        NaN   NaN   NaN   

Province_State                             
Admin2         Unassigned Washakie Weston  
2020-07-08       0.013158      NaN    NaN  
2020-07-09       0.006579      NaN    NaN  
2020-07-10       0.003290      NaN    NaN  

[3 rows x 3254 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20200710,AK,1323,140608,NaN,30.0,NaN,NaN,NaN,0.0,...,141931,0,0,7337a3e96732733e1dad90b6631e3e4316bba0ff,0,0,0,0,0,NaN
1,20200710,AL,50508,441708,NaN,1183.0,3089.0,NaN,888.0,NaN,...,492216,36,50,9e12c62a712d9f83add02bd471d6608745a2c76a,0,0,0,0,0,NaN
2,20200710,AR,26803,338609,NaN,402.0,1743.0,NaN,NaN,84.0,...,365412,4,38,3ebd2a7aee71250e30472d87c0a43986dfcd32aa,0,0,0,0,0,NaN
3,20200710,AS,0,816,NaN,NaN,NaN,NaN,NaN,NaN,...,816,0,0,004c2c497969b52749e6bc8d88e68f7e39c1818b,0,0,0,0,0,NaN
4,20200710,AZ,116892,552319,NaN,3432.0,5659.0,876.0,NaN,615.0,...,669211,44,133,2c23eaf50b677706200aa0be2e84a000193f759f,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20200710,AK,1323.0,140608.0,NaN,30.0,NaN,NaN,NaN,0.0,...,141931,0,0,7337a3e96732733e1dad90b6631e3e4316bba0ff,0,0,0,0,0,NaN
1,20200710,AL,50508.0,441708.0,NaN,1183.0,3089.0,NaN,888.0,NaN,...,492216,36,50,9e12c62a712d9f83add02bd471d6608745a2c76a,0,0,0,0,0,NaN
2,20200710,AR,26803.0,338609.0,NaN,402.0,1743.0,NaN,NaN,84.0,...,365412,4,38,3ebd2a7aee71250e30472d87c0a43986dfcd32aa,0,0,0,0,0,NaN
3,20200710,AS,0.0,816.0,NaN,NaN,NaN,NaN,NaN,NaN,...,816,0,0,004c2c497969b52749e6bc8d88e68f7e39c1818b,0,0,0,0,0,NaN
4,20200710,AZ,116892.0,552319.0,NaN,3432.0,5659.0,876.0,NaN,615.0,...,669211,44,133,2c23eaf50b677706200aa0be2e84a000193f759f,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,onVentilatorCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-07-10,AK,1323,140608,NaN,30.0,NaN,NaN,NaN,0.0,NaN,...,141931,0,0,7337a3e96732733e1dad90b6631e3e4316bba0ff,0,0,0,0,0,NaN
2020-07-10,AL,50508,441708,NaN,1183.0,3089.0,NaN,888.0,NaN,493.0,...,492216,36,50,9e12c62a712d9f83add02bd471d6608745a2c76a,0,0,0,0,0,NaN
2020-07-10,AR,26803,338609,NaN,402.0,1743.0,NaN,NaN,84.0,260.0,...,365412,4,38,3ebd2a7aee71250e30472d87c0a43986dfcd32aa,0,0,0,0,0,NaN
2020-07-10,AS,0,816,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,816,0,0,004c2c497969b52749e6bc8d88e68f7e39c1818b,0,0,0,0,0,NaN
2020-07-10,AZ,116892,552319,NaN,3432.0,5659.0,876.0,NaN,615.0,NaN,...,669211,44,133,2c23eaf50b677706200aa0be2e84a000193f759f,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,onVentilatorCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-07-10,AK,1323.0,140608.0,NaN,30.0,NaN,NaN,NaN,0.0,NaN,...,141931,0,0,7337a3e96732733e1dad90b6631e3e4316bba0ff,0,0,0,0,0,NaN
2020-07-10,AL,50508.0,441708.0,NaN,1183.0,3089.0,NaN,888.0,NaN,493.0,...,492216,36,50,9e12c62a712d9f83add02bd471d6608745a2c76a,0,0,0,0,0,NaN
2020-07-10,AR,26803.0,338609.0,NaN,402.0,1743.0,NaN,NaN,84.0,260.0,...,365412,4,38,3ebd2a7aee71250e30472d87c0a43986dfcd32aa,0,0,0,0,0,NaN
2020-07-10,AS,0.0,816.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,816,0,0,004c2c497969b52749e6bc8d88e68f7e39c1818b,0,0,0,0,0,NaN
2020-07-10,AZ,116892.0,552319.0,NaN,3432.0,5659.0,876.0,NaN,615.0,NaN,...,669211,44,133,2c23eaf50b677706200aa0be2e84a000193f759f,0,0,0,0,0,NaN


# TODO - CONTINUE FROM HERE